# 분류 알고리즘에 대한 탐색

- 고등학생과 미국 인구조사국 학생들이 손으로 쓴 70,000개의 작은 숫자 이미지를 모은 MNIST 데이터셋 사용
- 각 이미지에는 어떤 숫자를 나타내는지 레이블 되어있음
- MNIST 데이터셋 내려받기

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [51]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])